# A <strong>hypergeometric</strong> question
### By Hans Martin Aannestad

In my small department at UiB, consisting of 20 people, we are two named Hans Martin. After several confusing situations, Halvor said agitated: "What are the odds that two people in this room are actually called Hans Martin?"

The department's logician, Wim, quickly sketched out a solution from first principles, arriving at something resembling the hypergeometric distribution. Despite his effort, no one in the quite educated group was able to give an answer.

(According to https://www.ssb.no/befolkning/navn/statistikk/navn, there are 700)

### A machine learner's first thought: lets just simulate it:

In [119]:
import math
import random
from collections import Counter

pop = [0]*2500000  # assume a population of 2.5 million candidates
pop[:10] = [1]*700 # By Statistisk Sentralbyrå (SSB), about 700 with this name
random.shuffle(pop) 

num_specials = []

for i in range(10000000): # run 10 million experiments
    
    num_specials.append(sum(random.choices(pop,k=20))) # count number of occurrences in the office of 20

prob = Counter(num_specials)[2] / len(num_specials) # probability of (exactly) 2 occurrences
print('Number of 2 occurrences: '+str(Counter(num_specials)[2]))
print('The chance is about 1/' + str(round(1/prob)))

Number of 2 occurrences: 148
The chance is about 1/67568


### Alright, so it happened <strong>148</strong> times out of 10 million simulations. The chance is $ \frac{1}{67568} \approx 0.0015$%. That's pretty unlikely! Lets verify the simulation with Wim's fancy mathematics:

### Lets first implement our own binomial coefficent $ \binom{n}{k} = \frac{n!}{k!(n-k)!},$ in order to use it in $ P(k) = \frac{{n \choose k} {N - n\choose K - k}}{{N \choose K}} $ to get the exact probability.


In [121]:
def bnf(n,k): # binomial coefficient
    return math.factorial(n) / (math.factorial(k) * math.factorial(n-k))

# prob = (bnf(20,2) * bnf(2500000-20,700-2)) / bnf(2500000,700)  # Works, but slow

from scipy.stats import hypergeom  # Let's use something faster

prob = hypergeom(2500000, 20, 700).pmf(2)
print('The chance is exactly 1/' + str(round(1/prob)))

The chance is exactly 1/67567


### So the hypergeometric distribution with our example is:

### $ P(k=2) = \frac{{20 \choose 2} {2,500,000 - 20\choose 700 - 2}}{{2,500,000 \choose 700}} = \frac{1}{67567}$, in line with our simulation!

### A final comment: since <strong>more</strong> than 2 would also be confusing in the office, the probability is actually the cumulative:

In [120]:
prob1 = sum(hypergeom(2500000, 20, 700).pmf(range(2,20)))  # 2,3,...,20 (sum of the prob. density function)
prob2 = 1-hypergeom(2500000, 20, 700).cdf(1)  # Alternatively: 1 - P(k < 2) = 1 - P(0) + P(1), (by cumulative dist. function)
print('The chance of 2 or more is 1/' + str(round(1/prob2)))

The chance of 2 or more is 1/67424


### Let's check the number of occurences in our 10 million runs:

In [124]:
print(Counter(num_specials))

Counter({0: 9943967, 1: 55885, 2: 148})


### 3 or more did not occur in the simulation...

# Indeed, very <strong>hypergeometric!</strong>

The code can be found at: https://github.com/aannestad/quirks/blob/main/random_seating.ipynb